### !!! Before running the code below, please run the script "01_Database_CA2_Tan Thien Nguyen.sql" first !!!

### The Idea 
#Step 0: Create 3 raw tables and 3 views on MariaDB in script "01_Database_CA2_Tan Thien Nguyen.sql"
   - prime_minister_raw stores PM_No., PM_Name, URL, Born_Date, Died_Date, Start_Date, End_Date and Party information
   - prime_minister_province stores only PM_No., PM_Name and Province information
   - province_raw stores province master information
   - Three views (v_pm_province_master, v_prime_minister_master, v_province_master) are used to store transformed data
   
#Step 1: Find the tables which contain the information of PMs and provinces by using soup

#Step 2: Create function to get PMs' born date and died date based on URL

#Step 3: Declare SQL variables which are used for inserting data into database

#Step 4: Insert Data into Province Master table - province_raw

#Step 5: Insert Data into the table prime_minister_province

#Step 6: Biggest step - Scraping data of all prime ministers and Insert raw data into the table prime_minister_raw

#Step 7: Answer the questions by using DBcm to get data and pandas to display

----------------------

### Step 1: Find the tables which contain the information of PMs and provinces

In [ ]:
URL = "https://en.wikipedia.org/wiki/List_of_prime_ministers_of_Canada"

import gazpacho

from gazpacho import get
html = get(URL)

from gazpacho import Soup
soup = Soup(html)

tables = soup.find("table")
for i, t in enumerate(tables):
    if t.attrs["class"] == "wikitable":  # Looking for only the wikitables.
        rows = t.find("tr")
        # Check if we are getting back a Soup or a List.
        if isinstance(rows, list):
            print(i, len(rows))
        else:
            print(i, "Table has only one row - is it Soup?")

provinceall = tables[0]
prime = tables[1]
prime_tr = prime.find("tr")

### Step 2: Create function to get born date and died date based on URL

In [ ]:
camonth = ('JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER')

def getBornDiedDate(URL):
    html = get(URL)
    soup = Soup(html)
    tables = soup.find("table")
    information = tables[0].find("tr")
    if not isinstance(information, list):
        information = tables[1].find("tr")
    li = []
    
    # Looking for the the row containing born date and died date
    for i, t in enumerate(information):
        if t.find("th") != None:
            rows = t.find("th")
            if (not isinstance(rows,list) and (rows.text == "Born" or rows.text == "Died")):
                info = information[i].strip().upper()
                upperInfo = info.upper()
                # Extract born date from here
                for inmo in camonth: 
                    if upperInfo.find(inmo) > 1:
                        i = upperInfo.find(inmo)
                        shortInfo = upperInfo[i:]
                        blankspindex = shortInfo.find(" ")    #Find the nearest space with month name to find comma
                        commastr = shortInfo[:blankspindex+5] #Check comma nearest with month name to avoid get comma from other part, e.g. Ontario, Canada
                        commaindex = commastr.find(",")
                        if commaindex > 1:  # if True, type date is January 12, 1821 if False, type date is 12 January 1821
                            finaldate = shortInfo[:commaindex+6]
                        else:
                            finaldate = upperInfo[i-3:i].replace(")","") + shortInfo[:blankspindex+5]
                        li.append(finaldate.replace(",","").title())
    if len(li) < 2:
        li.append(None)
        if li[0][0].isnumeric() == False and li[0][1].isnumeric() == True:
            li[0] = li[0][1:]
    else:
        if li[0][0].isnumeric() == False and li[0][1].isnumeric() == True:
            li[0] = li[0][1:]
        if li[1][0].isnumeric() == False and li[1][1].isnumeric() == True:
            li[1] = li[1][1:]
    return (li)

### Step 3: Declare variables which are used for inserting data into database

In [ ]:
import DBcm

config = {
    'host':'127.0.0.1',
    'database':'canada_pm_tan',
    'user':'tan',
    'password':'123456'
}

SQL_Truncate_Prime = """
Truncate table prime_minister_raw;
"""

SQL_Insert_Prime = """
Insert into prime_minister_raw(pmno, name, url, birthdeath, pmborndate, pmdieddate, startoffice, endoffice, party)
values (%s,%s,%s,%s,%s,%s,%s,%s,%s);
"""

SQL_PM_Province_Truncate = """
Truncate table prime_minister_province;
"""

SQL_PM_Province = """
Insert into prime_minister_province(pmno, name, code, province)
values (%s,%s,%s,%s);
"""

SQL_Master_Province_Truncate = """
Truncate table province_raw;
"""

SQL_Master_Province = """
Insert into province_raw(code, province)
values (%s,%s);
"""

### Step 4: Insert Data into Province Master table

In [ ]:
pv = provinceall.find("tr")[2].find("td")[1].strip().replace(", ",",").split(",")

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_Master_Province_Truncate)
    
for p in pv:
    code = p[0:2]
    print(code)
    provincedetail = p[4:]
    print(provincedetail)
    with DBcm.UseDatabase(config) as cursor:
        cursor.execute(SQL_Master_Province,(code,provincedetail))

### Step 5: Insert Data into the table Prime_Minister_Province

In [ ]:
i = 0

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_PM_Province_Truncate)

for pt in (prime_tr[1:]):  # Start from detail rows
    rows = pt.find("td")               # Get all columns in the rows
    if isinstance(rows, list):    # if there is multiple columns, we need it, if only one columns, it means row for detail with colspan = 4
        if len(rows) == 8:  # for row contain PM infor
            tag = pt.find("td")[2].find("a")   # find column contain infor of Name, birth death and province
            if isinstance(tag, list):  # if have multiple tags, it means that it contain name and link for province, if not, it means it only contain name eg. John Abbolt
                i = i + 1
                pmNo = pt.find("td")[0].text.replace("(","").replace(")","")
                pmName = pt.find("td")[2].find("a")[0].attrs["title"]
                li = pt.find("td")[2].strip().split("MP")[1:]
                for province in li:
                    print(i)
                    print(pmNo)
                    print(pmName)
                    print(province)
                    if province.find(",") > 1 :
                        code = province[province.find(",")+2: province.find(",")+4]
                    else:
                        code = province
                    print(code)
                    with DBcm.UseDatabase(config) as cursor:
                        cursor.execute(SQL_PM_Province,(pmNo,pmName,code,province))

            else:
                i = i + 1
                pmNo = pt.find("td")[0].text.replace("(","").replace(")","")
                pmName = pt.find("td")[2].find("a").attrs["title"]
                pmProvince = pt.find("td")[2].strip().split(")")[1]
                print(i)
                print(pmNo)
                print(pmName)
                print(pmProvince)
                if pmProvince.find("for") > 1 :
                    code = pmProvince[pmProvince.find("for")+4: ]
                else:
                    code = "No Province"
                print(code)
                with DBcm.UseDatabase(config) as cursor:
                    cursor.execute(SQL_PM_Province,(pmNo,pmName,code,province))
            print("-------------------")

## Step 6: Biggest step - Scraping data of all prime ministers and Insert raw data into the table prime_minister_raw. This step usually takes 30s to complete

In [ ]:

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_Truncate_Prime)

i = 0

for pt in (prime_tr[1:]):  # start from detail rows
    rows = pt.find("td")               # how many columns the row has
    if isinstance(rows, list):    # if there is multiple columns, we need it, if only one columns, it means row for detail with colspan = 4
        if len(rows) == 8:  # for row contain PM infor
            tag = pt.find("td")[2].find("a")   # find column contain infor of Name, birth death and province
            if isinstance(tag, list):  # if have multiple tags, it means that it contain name and link for province, if not, it means it only contain name eg. John Abbolt
                i = i + 1
                pmNo = pt.find("td")[0].text.replace("(","").replace(")","")
                pmName = pt.find("td")[2].find("a")[0].attrs["title"]
                pmUrl = pt.find("td")[2].find("a")[0].attrs["href"]
                pmBirthDeath = pt.find("td")[2].find("small")[0].text.replace(chr(8211),"-")
                
                if pmNo == "20":
                    pmUrl = "/wiki/Jean_Chretien"
                fullURL = "https://en.wikipedia.org" + pmUrl
                pmBornDate = getBornDiedDate(fullURL)[0]
                pmDiedDate = getBornDiedDate(fullURL)[1]
                
                pmStart = pt.find("td")[3].strip().split(chr(8211))[0]
                
                pmEnd = pt.find("td")[3].strip().split(chr(8211))[1].replace("[*]","")
                start = pmEnd.find("/")
                end = pmEnd.find(" ")
                removest = pmEnd[start:end]
                pmEnd = pmEnd.replace(removest,"")
                if pmEnd.find("cumbent") > 1:
                    pmEnd = None
                
                pmParty = pt.find("td")[5].text
            else:
                i = i + 1
                pmNo = pt.find("td")[0].text.replace("(","").replace(")","")
                pmName = pt.find("td")[2].find("a").attrs["title"]
                pmUrl = pt.find("td")[2].find("a").attrs["href"]
                pmBirthDeath = pt.find("td")[2].find("small")[0].text.replace(chr(8211),"-")
                
                fullURL = "https://en.wikipedia.org" + pmUrl
                pmBornDate = getBornDiedDate(fullURL)[0]
                pmDiedDate = getBornDiedDate(fullURL)[1]
                
                pmStart = pt.find("td")[3].strip().split(chr(8211))[0]
                
                pmEnd = pt.find("td")[3].strip().split(chr(8211))[1].replace("[*]","")
                start = pmEnd.find("/")
                end = pmEnd.find(" ")
                removest = pmEnd[start:end]
                pmEnd = pmEnd.replace(removest,"")
                
                pmParty = pt.find("td")[5].text
        else: ## for row only containt term of office Robert Borden
            pmNo = pmNo
            pmName = pmName
            pmUrl = pmUrl
            pmBirthDeath = pmBirthDeath
            pmBornDate = pmBornDate
            pmDiedDate = pmDiedDate
            pmStart = pt.find("td")[0].strip().split(chr(8211))[0]
            pmEnd = pt.find("td")[0].strip().split(chr(8211))[1].replace("[*]","")
            pmParty = pt.find("td")[2].text
        print(i)
        print(pmNo)
        print(pmName)
        print("https://en.wikipedia.org" + pmUrl)
        print(pmBirthDeath)
        print(pmBornDate)
        print(pmDiedDate)
        print(pmStart)
        print(pmEnd)
        print(pmParty)
        print("-------------------")
        
        with DBcm.UseDatabase(config) as cursor:
            cursor.execute(SQL_Insert_Prime,(pmNo,pmName,pmUrl,pmBirthDeath,pmBornDate,pmDiedDate,pmStart,pmEnd,pmParty))

### Step 7: Answers

In [ ]:
import pandas as pd

#### (a) Which political party has produced the most individual prime ministers? [2]

In [ ]:
SQL_a = """
select party, count(name) as prime_minister_count 
from v_prime_minister_master group by party 
order by count(name) desc limit 1;
"""
with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_a)
    data = cursor.fetchall()
    labels = ('Party','PM_Count')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (b) Provide a list of prime ministers who served by province/territory. [3]

In [ ]:
SQL_b = """
select code as province_code, province, name from v_pm_province_master order by code;
"""
with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_b)
    data = cursor.fetchall()
    labels = ('Province Code','Province Name', 'Prime Minister')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (c) Which political party held the office of prime minister for the longest amount of overall time, and for how long? [3]

In [ ]:
SQL_c = """
select party, sum(held_day) as held_day_sum 
from ( select party, timestampdiff(day,startoffice, endoffice) as held_day from v_prime_minister_master
) h 
group by party 
order by held_day_sum desc limit 1;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_c)
    data = cursor.fetchall()
    labels = ('Party','Total of Held Days')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (d) Which individual politician held the office of prime minister for the longest amount of uninterrupted time, and for how long? [2]

In [ ]:
SQL_d = """
select pmno, name, startoffice, endoffice,
timestampdiff(day,startoffice,endoffice) as total_held_day,
concat(
case when held_year = 0 then '' else held_year end, 
case when held_year = 0 then '' when held_year = 1 then ' year ' when held_year > 1 then ' years ' end, 
case when held_month = 0 then '' else held_month end, 
case when held_month = 0 then '' when held_month = 1 then ' month ' when held_month > 1 then ' months ' end,
case when held_day = 0 then '' else held_day end, 
case when held_day = 0 then '' when held_day = 1 then ' day ' when held_day > 1 then ' days ' end
) as Detail
from 
(
select pmno, name, startoffice, endoffice,held_year,held_month,
timestampadd(month,held_month,year_day) as month_day,
timestampdiff(day,timestampadd(month,held_month,year_day),endoffice) as held_day
from (
select pmno, name, startoffice, endoffice, held_year,
timestampadd(year,held_year,startoffice) as year_day,
timestampdiff(month,timestampadd(year,held_year,startoffice),endoffice) as held_month
from (
select pmno, name, startoffice, endoffice, 
timestampdiff(year,startoffice,endoffice) as held_year 
from v_prime_minister_master order by held_year desc limit 1
) h1
) h2
) h3;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_d)
    data = cursor.fetchall()
    labels = ('PM No.','Prime Minister','Start Date','End Date', 'Total of Held Days', 'Detail View')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (e) Which individual politician has held the office of prime ministers for the longest amount of overall time? [1]

In [ ]:
SQL_e = """
select pmno, name, sum(held_day) as held_day_sum, count(pmno) as term 
from ( select pmno, name, timestampdiff(day,startoffice,endoffice) as held_day from v_prime_minister_master) h 
group by pmno, name order by held_day_sum desc limit 1;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_e)
    data = cursor.fetchall()
    labels = ('PM No.','Prime Minister', 'Total of Held Days','No. of Term')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (f) Which individual politician held the office of prime minister for the shortest amount of overall time, and for how long? [1]

In [ ]:
SQL_f = """
select pmno, name, sum(held_day) as held_day_sum 
from (select pmno, name, timestampdiff(day,startoffice,endoffice) as held_day from v_prime_minister_master 
) h 
group by pmno, name 
order by held_day_sum asc limit 1;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_f)
    data = cursor.fetchall()
    labels = ('PM No.','Prime Minister', 'Total of Held Days')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (g) What age was each prime minister on the day they assumed office? [4]

In [ ]:
SQL_g = """
select pmno, name, pmborndate, startoffice, timestampdiff(year,pmborndate,startoffice) as age 
from v_prime_minister_master;
"""
with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_g)
    data = cursor.fetchall()
    labels = ('PM No.','Prime Minister', 'Born Date', 'Start Date','Age at the Start Date')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (h) On the last day of their term of office, which politician was the oldest? [1]

In [ ]:
SQL_h = """
select pmno, name, pmborndate, endoffice, timestampdiff(year,pmborndate,endoffice) as age 
from v_prime_minister_master 
order by age desc limit 1;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_h)
    data = cursor.fetchall()
    labels = ('PM No.','Prime Minister', 'Born Date', 'End Date','Age')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (i) In your view, and based on statistics calculated from your scraped data, which political party is the most successful? Show and describe your reasoning. [3]

#### >>> Liberal Party is the most successful political party in Canada because of the following reasons:

#### (i1.1) - Liberal Party has proceduced the most individual prime minister (13). This number is nearly double the second one (7)

In [ ]:
SQL_i11 = """
select party, count(name) as prime_minister_count 
from v_prime_minister_master 
group by party 
order by count(name) desc;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_i11)
    data = cursor.fetchall()
    labels = ('Party','Total of PMs')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (i1.2) - and nearly equal total of the PMs produced by other parties (13 vs 16)

In [ ]:
SQL_i12 = """
select party, count(name) as prime_minister_count 
from (
select case when party = 'Liberal Party' then 'Liberal Party' else 'Other Parties' end as party, name 
from v_prime_minister_master 
) h 
group by party order by count(name);
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_i12)
    data = cursor.fetchall()
    labels = ('Party','Total of PMs')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (i2.1) - Liberal Party has held the office of PM for the longest amount of overall time (more than 32500 days), nearly triple the second one (around 11600 days)


In [ ]:
SQL_i21 = """
select party, sum(held_day) as held_day_sum 
from (
select party, timestampdiff(day,startoffice, endoffice) as held_day from v_prime_minister_master 
) h 
group by party 
order by held_day_sum desc;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_i21)
    data = cursor.fetchall()
    labels = ('Party','Total of Held Days')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (i2.2.) - and especially, it is much larger than total of the days held by other parties

In [ ]:
SQL_i22 = """
select party, sum(held_day) as held_day_sum 
from (
select case when party = 'Liberal Party' then 'Liberal Party' 
else 'Other Parties' end as party, 
timestampdiff(day,startoffice, endoffice) as held_day 
from v_prime_minister_master 
) h group by party order by held_day_sum desc;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_i22)
    data = cursor.fetchall()
    labels = ('Party','Total of Held Days')
    df = pd.DataFrame.from_records(data,columns = labels)
df

#### (i3) - the last reason, 3 out of 4 last prime ministers come from Liberal Party

In [ ]:
SQL_i3 = """
select pmno, name, party, startoffice, 
case when endoffice = current_date then 'Incumbent' 
else endoffice end as endoffice 
from v_prime_minister_master 
order by pmno desc limit 4;
"""

with DBcm.UseDatabase(config) as cursor:
    cursor.execute(SQL_i3)
    data = cursor.fetchall()
    labels = ('PM No.','Name', 'Party', 'Start Date', 'End Office')
    df = pd.DataFrame.from_records(data,columns = labels)
df